In [4]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Load the data
batsmen_runs_df = pd.read_csv('batsmen_runs.csv')

# Select the relevant features and target variable
features = ['batting_team', 'bowling_team', 'venue', 'inning']
target = 'total_runs'

X = batsmen_runs_df[features]
y = batsmen_runs_df[target]

# One-hot encode categorical features
X_encoded = pd.get_dummies(X)

# Normalize the numerical features
scaler = MinMaxScaler()
X_encoded[['inning']] = scaler.fit_transform(X_encoded[['inning']])

# Print the list of all teams
all_teams = X['batting_team'].unique()
print("Available teams:")
for team in all_teams:
    print(team)

# Select team1
team1 = input("Enter team1: ")

# Filter players who played for team1
team1_players = batsmen_runs_df[batsmen_runs_df['batting_team'] == team1]['batsman'].unique()

print("Players who played for", team1 + ":")
for player in team1_players:
    print(player)

# Select a player from team1
player1 = input("Select a player from the above list: ")

# List teams played against team1
team1_opposition_teams = batsmen_runs_df[batsmen_runs_df['batting_team'] == team1]['bowling_team'].unique()

print("Teams played against", team1 + ":")
for team in team1_opposition_teams:
    print(team)

# Select team2 (opposition team)
team2 = input("Enter the opposition team: ")

# Filter data for team1 and team2
filtered_data = batsmen_runs_df[(batsmen_runs_df['batting_team'] == team1) & (batsmen_runs_df['bowling_team'] == team2)]

if filtered_data.empty:
    print("No records found for the specified team and opposition team.")
    exit()

# List venues for team1 and team2
team1_team2_venues = filtered_data['venue'].unique()

print("Venues where", team1, "played against", team2 + ":")
for venue in team1_team2_venues:
    print(venue)

# Select the venue
venue = input("Enter the venue: ")

# List innings
innings = filtered_data['inning'].unique()
print("Available innings:")
for inning in innings:
    print(inning)

# Select the inning
inning = int(input("Enter the inning: "))

# Filter data based on selected venue, inning, and pitch condition
filtered_data = filtered_data[(filtered_data['venue'] == venue) & (filtered_data['inning'] == inning)]

if filtered_data.empty:
    print("No records found for the specified venue, inning.")
    exit()

# Prepare the data for modeling
X_filtered = filtered_data[features]
y_filtered = filtered_data[target]

# One-hot encode categorical features for filtered data
X_filtered_encoded = pd.get_dummies(X_filtered)

# Ensure consistent feature order with training data
missing_columns = set(X_encoded.columns) - set(X_filtered_encoded.columns)
for column in missing_columns:
    X_filtered_encoded[column] = 0
X_filtered_encoded = X_filtered_encoded[X_encoded.columns]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered_encoded, y_filtered, test_size=0.2, random_state=42)

# Train and test Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_rmse = mean_squared_error(y_test, rf_predictions, squared=False)

# Train and test SVM model
svm_model = SVR(C=1.0, epsilon=0.2)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_rmse = mean_squared_error(y_test, svm_predictions, squared=False)

# Train and test XGBoost model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)


# List pitch conditions
pitch_conditions = ['wet pitch', 'dry pitch']
print("Available pitch conditions:")
for condition in pitch_conditions:
    print(condition)

# Select the pitch condition
pitch_condition = input("Select the pitch condition: ")

# Print the root mean squared error for each model
print("Random Forest RMSE:", rf_rmse)
print("SVM RMSE:", svm_rmse)
print("XGBoost RMSE:", xgb_rmse)

# Predict runs for the given inputs
input_data = pd.DataFrame([[team1, team2, venue, inning]], columns=features)
input_data_encoded = pd.get_dummies(input_data)

# Ensure consistent feature order with training data
missing_columns = set(X_encoded.columns) - set(input_data_encoded.columns)
for column in missing_columns:
    input_data_encoded[column] = 0
input_data_encoded = input_data_encoded[X_encoded.columns]

rf_predicted_runs = rf_model.predict(input_data_encoded)
svm_predicted_runs = svm_model.predict(input_data_encoded)
xgb_predicted_runs = xgb_model.predict(input_data_encoded)

# Adjust predicted runs based on pitch condition
if pitch_condition == 'wet pitch':
    rf_predicted_runs *= 1.2
    svm_predicted_runs *= 1.2
    xgb_predicted_runs *= 1.2

print("Predicted runs (Random Forest):", rf_predicted_runs[0])
print("Predicted runs (SVM):", svm_predicted_runs[0])
print("Predicted runs (XGBoost):", xgb_predicted_runs[0])


Available teams:
Royal Challengers Bangalore
Kolkata Knight Riders
Kings XI Punjab
Chennai Super Kings
Rajasthan Royals
Delhi Daredevils
Mumbai Indians
Deccan Chargers
Kochi Tuskers Kerala
Pune Warriors
Sunrisers Hyderabad
Rising Pune Supergiants
Gujarat Lions
Enter team1: Royal Challengers Bangalore
Players who played for Royal Challengers Bangalore:
AA Noffke
B Akhil
CL White
JH Kallis
MV Boucher
P Kumar
R Dravid
SB Joshi
V Kohli
W Jaffer
Z Khan
LRPL Taylor
S Chanderpaul
R Vinay Kumar
B Chipli
DW Steyn
Misbah-ul-Haq
A Kumble
DT Patil
J Arunkumar
SP Goswami
Abdur Razzak
JD Ryder
KP Pietersen
RV Uthappa
R Bishnoi
KV Sharma
Pankaj Singh
MK Pandey
RE van der Merwe
KP Appanna
D du Preez
EJG Morgan
A Mithun
S Sriram
AB de Villiers
AUK Pathan
MA Agarwal
SS Tiwary
TM Dilshan
CA Pujara
DL Vettori
JJ van der Wath
R Ninan
S Aravind
CH Gayle
J Syed Mohammad
M Kaif
KB Arun Karthik
LA Pomersbach
CK Langeveldt
AB McDonald
HV Patel
RR Bhatkal
M Muralitharan
P Parameswaran
DT Christian
KK Nair
JD Una